<a href="https://colab.research.google.com/github/tanishqtomar44/PRODIGY_GA_01/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes pandas torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Disable WandB Logging
os.environ["WANDB_DISABLED"] = "true"
# List of uploaded files
files = [
    "/content/Animals.csv",
    "/content/Books.csv",
    "/content/Climate.csv",
    "/content/Environment.csv",
    "/content/Friends.csv",
    "/content/Hospital.csv",
    "/content/Movies.csv",
    "/content/Religion.csv",
    "/content/School.csv",
    "/content/Space.csv"
]
df_list = [pd.read_csv(file) for file in files]
df = pd.concat(df_list, ignore_index=True)
df.head()



,Input Text,Target Text
0,Lions are known for their,majestic mane
1,Elephants are the largest,land animals
2,Dolphins are highly,intelligent marine mammals
3,Tigers are famous for their,striking stripes
4,Giraffes have a long,neck


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
df_list = [pd.read_csv(file) for file in files]
df = pd.concat(df_list, ignore_index=True)

#to verify displaying few line
print("DataFrame Columns:", df.columns)
print(df.head())

DataFrame Columns: Index(['Input Text', 'Target Text'], dtype='object')
                    Input Text                 Target Text
0    Lions are known for their               majestic mane
1    Elephants are the largest                land animals
2          Dolphins are highly  intelligent marine mammals
3  Tigers are famous for their            striking stripes
4         Giraffes have a long                        neck


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized = tokenizer(examples["Input Text"], padding="max_length", truncation=True)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Convert DataFrame to Hugging Face Dataset and tokenize it
dataset = Dataset.from_pandas(df)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["Input Text"])
tokenized_datasets.set_format("torch")

print("Tokenized dataset:")
print(tokenized_datasets)

# Load Pre-trained GPT-2 Model
model = GPT2LMHeadModel.from_pretrained("gpt2")
    #Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated parameter name
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    report_to=[],  # Disables logging to WandB
)

# Initializing Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Training the Model
trainer.train()

# Saving the Fine-Tuned Model
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

# Generating and reloading  Text with the Fine-Tuned Model and tokenizer
from transformers import pipeline

model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2")

# to create a text generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "Once upon a time"
generated_text = text_generator(prompt, max_length=50, num_return_sequences=1)
print("Generated Text:\n", generated_text[0]["generated_text"])

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

Tokenized dataset:
Dataset({
    features: ['Target Text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 509
})


<ipython-input-6-1ae3ae75d0fa>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,0.010107
2,0.123800,0.008203
3,0.123800,0.007782


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Text:
 Once upon a time your
